In [1]:
#import multiple files with names as Kickstarter*.csv into pandas dataframe (data used 15 Aug 2019)

from glob import glob
import pandas as pd
from pandas.io.json import json_normalize
import json as json
import numpy as np
from datetime import datetime

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

#just replace the path parts
#glob import all files starting with Kickstarter in the folder (the folder i used is the 15 Aug 2019)

filenames = glob(r"/mnt/CS3244_Proj_Data/Kickstarter_201*/Kickstarter*.csv")

df_chunks = []
for f in filenames:
    for chunk in pd.read_csv(f, encoding='utf-8', chunksize=100000):
        chunk['launched_at_utc'] = chunk['launched_at'].apply(datetime.utcfromtimestamp)
        chunk['deadline_utc'] = chunk['deadline'].apply(datetime.utcfromtimestamp)
        chunk['created_at_utc'] = chunk['created_at'].apply(datetime.utcfromtimestamp)
        chunk['goal'] = round(chunk['goal'] * chunk['static_usd_rate'], 2)
        x1 = json_normalize(chunk['category'].apply(json.loads)).add_prefix('category_')
        chunk = pd.concat([chunk, x1], axis = 1)
        df = chunk[['disable_communication', 'country', 'currency', 'goal', 'category_id', 'deadline_utc', 'launched_at_utc', 'state']]
        #pick the relevant data and drop all irrelevant data 

        #feature 1: duration of project 
        df['duration'] = (df['deadline_utc'] - df['launched_at_utc']).dt.days

        #feature 2: month of launch
        df['launch_month'] = df['launched_at_utc'].dt.month

        #change state to numerical 
        df['state'] = df['state'].map({'successful': 1, 'failed': 0 })

        #drop columns
        df = df.drop(columns = ['launched_at_utc', 'deadline_utc'])
        df = df.dropna() #drop null rows
        columns = df.columns.tolist()

        columns = columns[-2:] + columns[:-2]
        df_chunks.append(df[columns])
        

df = pd.concat(df_chunks, axis = 0, ignore_index = True).reset_index()
df = df.drop(columns = ['index'])
success_length = df.loc[df['state'] == 1].shape[0]
failure_length = df.shape[0] - success_length
sample_size = min(success_length, failure_length) // 5
print(sample_size)
# Get equal number of success and failure cases
df = pd.concat([df.loc[df['state'] == 1].sample(n=sample_size), df.loc[df['state'] == 0].sample(n=sample_size)])
print(df.columns.tolist())
print(df.shape)

/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

328902
['duration', 'launch_month', 'disable_communication', 'country', 'currency', 'goal', 'category_id', 'state']
(657804, 8)


In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

print(df.columns.tolist())
print(df.iloc[0].values)
X = np.array(df.iloc[:,:-1].values)
y = np.array(df.iloc[:,-1].values)

label_encoder_month_of_launch = LabelEncoder()
X[:, 1] = label_encoder_month_of_launch.fit_transform(X[:, 1])

label_encoder_disable_com = LabelEncoder()
X[:, 2] = label_encoder_disable_com.fit_transform(X[:, 2])

label_encoder_country = LabelEncoder()
X[:, 3] = label_encoder_country.fit_transform(X[:, 3])

label_encoder_currency = LabelEncoder()
X[:, 4] = label_encoder_currency.fit_transform(X[:, 4])

label_encoder_category = LabelEncoder()
X[:, 6] = label_encoder_category.fit_transform(X[:, 6])

one_hot_encoding = OneHotEncoder(categorical_features=[1, 2, 3, 4, 6])
X = one_hot_encoding.fit_transform(X).toarray()

['duration', 'launch_month', 'disable_communication', 'country', 'currency', 'goal', 'category_id', 'state']
[31 8 False 'US' 'USD' 1000.0 47 1.0]


/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape)

(526243, 221) (131561, 221)


In [4]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential()

input_dim = int(X_train.shape[1])
output_dim = input_dim // 3

print('Input dim: ', input_dim)
print('Output dim: ', output_dim)

#input layer
classifier.add(Dense(units=output_dim, activation='relu', input_dim=input_dim))
classifier.add(Dropout(p=0.1))

# hidden layer
classifier.add(Dense(units=output_dim, activation='relu'))
classifier.add(Dropout(rate=0.1))
classifier.add(Dense(units=output_dim, activation='relu'))
classifier.add(Dropout(rate=0.1))

# output layer
classifier.add(Dense(units=1, activation='sigmoid'))

# compile ann
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit ANN
classifier.fit(X_train, y_train, batch_size=30, epochs=40)

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Input dim:  221
Output dim:  73


/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  app.launch_new_instance()


Epoch 1/40
526243/526243 [==============================] - 51s 97us/step - loss: 0.4715 - acc: 0.7570
Epoch 2/40
526243/526243 [==============================] - 50s 95us/step - loss: 0.4602 - acc: 0.7626
Epoch 3/40
526243/526243 [==============================] - 54s 102us/step - loss: 0.4559 - acc: 0.7647
Epoch 4/40
526243/526243 [==============================] - 52s 99us/step - loss: 0.4532 - acc: 0.7664
Epoch 5/40
526243/526243 [==============================] - 51s 98us/step - loss: 0.4508 - acc: 0.7670
Epoch 6/40
526243/526243 [==============================] - 50s 94us/step - loss: 0.4487 - acc: 0.7688
Epoch 7/40
526243/526243 [==============================] - 51s 97us/step - loss: 0.4470 - acc: 0.7702
Epoch 8/40
526243/526243 [==============================] - 52s 98us/step - loss: 0.4455 - acc: 0.7708
Epoch 9/40
526243/526243 [==============================] - 46s 88us/step - loss: 0.4442 - acc: 0.7719
Epoch 10/40
526243/526243 [==============================] - 46s 87us/st

In [6]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score
accuracy_score = accuracy_score(y_test, y_pred)
confusion_matrix = confusion_matrix(y_test, y_pred) / len(y_pred)
roc_score = roc_auc_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)
print("Accuracy score of: ", accuracy_score)
print("False Positive rate of: ", confusion_matrix[0, 1])
print("False Negative rate of: ", confusion_matrix[1, 0])
print("ROC AUC Score of: ", roc_score)
print("F1 Score of: ", f1_score)

Accuracy score of:  0.7818426433365511
False Positive rate of:  0.06928345026261581
False Negative rate of:  0.14887390640083306
ROC AUC Score of:  0.7819745276182406
F1 Score of:  0.7634058478760849
